In [ ]:
# ==============================================================================
# CELL 1: INSTALL DEPENDENCIES
# ==============================================================================
!pip install -q -U langchain-community langchain-huggingface sentence-transformers faiss-cpu langchain openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# ==============================================================================
# CELL 1: IMPORTS & CONFIGURATION
# ==============================================================================
import os
import json
import pandas as pd
from google.colab import drive
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from sentence_transformers import CrossEncoder
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Mount Google Drive
drive.mount('/content/drive')

# Configuration Paths
BASE_DIR = "/content/drive/My Drive/Capstone Project/FINAL_FOLDER"
INPUT_CHUNKS_DIR = os.path.join(BASE_DIR, "Maitri/retrieval_results_new")
INPUT_TEXT_DIR = os.path.join(BASE_DIR, "Text Extraction/text_extracted")
METADATA_CSV = os.path.join(BASE_DIR, "Text Extraction/metadata_new_final_v1.csv")
OUTPUT_DIR = os.path.join(BASE_DIR, "Summary and Eval/FINAL_APPROACH_v4/results")

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Chunks Directory: {INPUT_CHUNKS_DIR}")
print(f"Text Directory: {INPUT_TEXT_DIR}")
print(f"Metadata CSV: {METADATA_CSV}")
print(f"Output Directory: {OUTPUT_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Chunks Directory: /content/drive/My Drive/Capstone Project/FINAL_FOLDER/Maitri/retrieval_results_new
📂 Text Directory: /content/drive/My Drive/Capstone Project/FINAL_FOLDER/Text Extraction/text_extracted
📄 Metadata CSV: /content/drive/My Drive/Capstone Project/FINAL_FOLDER/Text Extraction/metadata_new_final_v1.csv
📂 Output Directory: /content/drive/My Drive/Capstone Project/FINAL_FOLDER/Summary and Eval/FINAL_APPROACH_v4/results


In [ ]:
# List available chunk files
chunk_files = [f for f in os.listdir(INPUT_CHUNKS_DIR) if f.startswith("CHUNKS_")]
print(f"\nFound {len(chunk_files)} chunk files:")
for f in chunk_files:
    print(f"   - {f}")


✅ Found 8 chunk files:
   - CHUNKS_How_have_the_rates_of_liver_transplantation_for_HC.json
   - CHUNKS_What_are_factors_associated_with_increased_use_of_.json
   - CHUNKS_How_has_the_evidence_changed_regarding_acceptable_.json
   - CHUNKS_How_has_machine_perfusion_changed_risk_thresholds_.json
   - CHUNKS_What_is_the_impact_of_macro_microsteatosis_on_post.json
   - CHUNKS_What_are_the_current_research_knowledge_gaps_in_st.json
   - CHUNKS_Which_technologies_show_the_strongest_publication_.json
   - CHUNKS_How_have_innovations___impact_of_policies_spread_t.json


In [ ]:
!ls -ltr "/content/drive/My Drive/Capstone Project/FINAL_FOLDER/Maitri/retrieval_results_new"

total 325
-rw------- 1 root root  2741 Nov 25 18:26 PAPERS_How_have_the_rates_of_liver_transplantation_for_HC.json
-rw------- 1 root root 40447 Nov 25 18:26 CHUNKS_How_have_the_rates_of_liver_transplantation_for_HC.json
-rw------- 1 root root  2642 Nov 25 18:27 PAPERS_What_are_factors_associated_with_increased_use_of_.json
-rw------- 1 root root 37460 Nov 25 18:27 CHUNKS_What_are_factors_associated_with_increased_use_of_.json
-rw------- 1 root root  2512 Nov 25 18:29 PAPERS_How_has_the_evidence_changed_regarding_acceptable_.json
-rw------- 1 root root 36074 Nov 25 18:29 CHUNKS_How_has_the_evidence_changed_regarding_acceptable_.json
-rw------- 1 root root  2650 Nov 25 18:30 PAPERS_How_has_machine_perfusion_changed_risk_thresholds_.json
-rw------- 1 root root 35648 Nov 25 18:30 CHUNKS_How_has_machine_perfusion_changed_risk_thresholds_.json
-rw------- 1 root root  2815 Nov 25 18:31 PAPERS_What_is_the_impact_of_macro_microsteatosis_on_post.json
-rw------- 1 root root 38290 Nov 25 18:31 CHU

In [ ]:
pip install -U bitsandbytes -q

In [ ]:
# ==============================================================================
# CELL 3: LOAD ZEPHYR MODEL FOR SUMMARIZATION
# ==============================================================================
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "HuggingFaceH4/zephyr-7b-beta"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=True
)

print("⏳ Creating text generation pipeline...")
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1500,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    repetition_penalty=1.15
)

print("Zephyr model loaded successfully!")

⏳ Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


⏳ Loading model with 8-bit quantization...


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


⏳ Creating text generation pipeline...
✅ Zephyr model loaded successfully!


In [ ]:
# ==============================================================================
# CELL 4: HELPER FUNCTIONS - LOAD CHUNKS AND PAPERS
# ==============================================================================

def load_chunks_from_json(chunk_file_path):
    """Load top 10 chunks from retrieval results"""
    with open(chunk_file_path, 'r') as f:
        chunks = json.load(f)
    return chunks

def load_papers_from_json(paper_file_path):
    """Load top 10 papers from retrieval results"""
    with open(paper_file_path, 'r') as f:
        papers = json.load(f)
    return papers

def get_full_paper_text(filename, text_dir):
    """Load full paper text from filename"""
    file_path = os.path.join(text_dir, filename)
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read()
    return None

print("Helper functions defined!")

Helper functions defined!


In [ ]:
# ==============================================================================
# CELL 5: SUMMARIZATION FUNCTIONS
# ==============================================================================

def generate_summary(context_text, query, max_input_words=3000):
    """Generate query-focused summary using your v3 prompt."""
    words = context_text.split()
    if len(words) > max_input_words:
        beginning = ' '.join(words[:int(max_input_words * 0.7)])
        end = ' '.join(words[-int(max_input_words * 0.3):])
        truncated_text = beginning + "\n\n[...content truncated...]\n\n" + end
    else:
        truncated_text = context_text

    prompt = f"""<|system|>
You are a specialized medical research assistant. Your task is to provide a query-focused summary of the provided text.

CRITICAL INSTRUCTIONS:
1. ANSWER THE QUERY: Your summary MUST directly answer the user's query. All other information is secondary.
2. STAY FAITHFUL: You MUST NOT include any information that is not explicitly present in the provided text.
3. NO HALLUCINATIONS: If the provided text does not contain the answer to the query, you MUST state: "The provided context does not contain the answer to this query."
4. Keep the summary between 150-250 words.
5. Use factual, objective language.

</s>
<|user|>
Based ONLY on the following extracted context from a research paper, write a concise, professional summary that specifically addresses this query: "{query}"

CONTEXT:
{truncated_text}

SUMMARY:
</s>
<|assistant|>"""

    outputs = text_generator(prompt, return_full_text=False)
    summary = outputs[0]['generated_text'].strip()
    summary = summary.replace("</s>", "").replace("<|assistant|>", "").strip()

    return summary

print("Summarization function defined!")

Summarization function defined!


In [ ]:
# ==============================================================================
# CELL 6: MAIN PROCESSING FUNCTION
# ==============================================================================

def process_query_results(chunk_file, paper_file, query, text_dir):
    """
    Process a query's results: load chunks/papers and generate summaries

    Args:
        chunk_file: Path to CHUNKS_*.json file
        paper_file: Path to PAPERS_*.json file
        query: The research question (string)
        text_dir: Directory containing full paper texts

    Returns:
        chunk_result, paper_results
    """
    print(f"\n{'='*80}")
    print(f"PROCESSING QUERY: {query}")
    print('='*80)

    # -------------------------------------------------------------------------
    # 1. LOAD DATA
    # -------------------------------------------------------------------------
    print("\nLoading chunks and papers...")
    chunks = load_chunks_from_json(chunk_file)
    papers = load_papers_from_json(paper_file)

    print(f"Loaded {len(chunks)} chunks and {len(papers)} papers")

    # -------------------------------------------------------------------------
    # 2. CHUNK SUMMARY
    # -------------------------------------------------------------------------
    print("\nGenerating chunk-based summary...")
    combined_chunks = "\n\n---\n\n".join([c['text'] for c in chunks])
    chunk_summary = generate_summary(combined_chunks, query)

    # Extract DOIs and prepare contexts for RAGAS
    chunk_dois = list(set([c['doi'] for c in chunks if c.get('doi') and c['doi'] != 'Unknown DOI']))
    chunk_contexts_list = [c['text'] for c in chunks]

    chunk_result = {
        'query': query,
        'summary_type': 'CHUNKS_ONLY',
        'summary': chunk_summary,
        'summary_word_count': len(chunk_summary.split()),
        'num_chunks': len(chunks),
        'cited_dois': chunk_dois,
        'contexts': chunks,
        'reference_text': combined_chunks,  # For traditional evaluation
        'retrieved_contexts': chunk_contexts_list  # For RAGAS
    }

    print(f"Chunk summary: {len(chunk_summary.split())} words")
    print(f"\nSUMMARY:\n{chunk_summary}")

    # -------------------------------------------------------------------------
    # 3. PAPER SUMMARIES
    # -------------------------------------------------------------------------
    print(f"\nGenerating summaries for {len(papers)} papers...")
    paper_summaries = []

    for i, paper in enumerate(papers, 1):
        print(f"\n  Paper {i}/{len(papers)}: {paper['title'][:60]}...")

        # Load full text
        full_text = get_full_paper_text(paper['filename'], text_dir)

        if full_text:
            print(f"    Loaded ({len(full_text.split())} words), generating summary...")
            summary = generate_summary(full_text, query)

            paper_summaries.append({
                'rank': paper['rank'],
                'title': paper['title'],
                'doi': paper['doi'],
                'year': paper['year'],
                'filename': paper['filename'],
                'reranker_score': paper['best_chunk_score'],
                'summary': summary,
                'summary_word_count': len(summary.split()),
                'original_word_count': len(full_text.split()),
                'reference_text': full_text,  # For traditional evaluation
                'retrieved_contexts': [full_text]  # For RAGAS
            })

            print(f"    Done ({len(summary.split())} words)")
        else:
            print(f"    Text file not found")

    paper_results = {
        'query': query,
        'summary_type': 'FULL_PAPERS',
        'num_papers': len(paper_summaries),
        'papers': paper_summaries
    }

    print(f"\nGenerated {len(paper_summaries)} paper summaries")

    # Display summaries
    for paper in paper_summaries:
        print(f"\n{'─'*80}")
        print(f"#{paper['rank']}: {paper['title'][:60]}...")
        print(f"DOI: {paper['doi']}")
        print(f"\n{paper['summary'][:200]}...")

    return chunk_result, paper_results

print("Main processing function defined!")

Main processing function defined!


In [ ]:
# ==============================================================================
# CELL 7: RUN THE PIPELINE
# ==============================================================================

# Input your query and corresponding files
query = input("Enter your research question: ")

# Find matching chunk and paper files
# List available files to help you choose
print("\nAvailable chunk files:")
chunk_files = sorted([f for f in os.listdir(INPUT_CHUNKS_DIR) if f.startswith("CHUNKS_")])
for i, f in enumerate(chunk_files, 1):
    print(f"  {i}. {f}")

chunk_file_name = input("\nEnter chunk filename (e.g., CHUNKS_How_have_the_rates_of_liver_transplantation_for_HC.json): ")
paper_file_name = chunk_file_name.replace("CHUNKS_", "PAPERS_")

# Build full paths
chunk_file_path = os.path.join(INPUT_CHUNKS_DIR, chunk_file_name)
paper_file_path = os.path.join(INPUT_CHUNKS_DIR, paper_file_name)

# Verify files exist
if not os.path.exists(chunk_file_path):
    print(f"Error: Chunk file not found: {chunk_file_path}")
elif not os.path.exists(paper_file_path):
    print(f"Error: Paper file not found: {paper_file_path}")
else:
    print(f"\nFiles found:")
    print(f"   Chunks: {chunk_file_name}")
    print(f"   Papers: {paper_file_name}")

    # Process the query
    chunk_result, paper_results = process_query_results(
        chunk_file=chunk_file_path,
        paper_file=paper_file_path,
        query=query,
        text_dir=INPUT_TEXT_DIR
    )

    print("\n" + "="*80)
    print("SUMMARIZATION COMPLETE!")
    print("="*80)

Enter your research question: How have innovations / impact of policies spread through the literature over time?

📂 Available chunk files:
  1. CHUNKS_How_has_machine_perfusion_changed_risk_thresholds_.json
  2. CHUNKS_How_has_the_evidence_changed_regarding_acceptable_.json
  3. CHUNKS_How_have_innovations___impact_of_policies_spread_t.json
  4. CHUNKS_How_have_the_rates_of_liver_transplantation_for_HC.json
  5. CHUNKS_What_are_factors_associated_with_increased_use_of_.json
  6. CHUNKS_What_are_the_current_research_knowledge_gaps_in_st.json
  7. CHUNKS_What_is_the_impact_of_macro_microsteatosis_on_post.json
  8. CHUNKS_Which_technologies_show_the_strongest_publication_.json

Enter chunk filename (e.g., CHUNKS_How_have_the_rates_of_liver_transplantation_for_HC.json): CHUNKS_How_have_innovations___impact_of_policies_spread_t.json

✅ Files found:
   Chunks: CHUNKS_How_have_innovations___impact_of_policies_spread_t.json
   Papers: PAPERS_How_have_innovations___impact_of_policies_spread_t.j

In [ ]:
# !pip install -q -U langchain langchain-community langchain-huggingface \
#                  sentence-transformers faiss-cpu transformers accelerate \
#                  bitsandbytes torch datasets rouge-score bert-score nltk \
#                  scikit-learn openpyxl pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 142.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 156.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.1 MB/s eta 0:00:00
  

In [ ]:
# ==============================================================================
# CELL 8: EVALUATION FUNCTIONS - TRADITIONAL METRICS
# ==============================================================================

import nltk
nltk.download('punkt', quiet=True)

from rouge_score import rouge_scorer
from bert_score import score as bert_score
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

print("Loading embedding model for evaluation...")
eval_embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def compute_rouge_scores(summary, reference):
    """Compute ROUGE scores."""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, summary)
    return {
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rouge2_precision': scores['rouge2'].precision,
        'rouge2_recall': scores['rouge2'].recall,
        'rougeL_f1': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall,
    }

def compute_semantic_similarity(summary, reference, embed_model):
    """Compute semantic similarity using embeddings."""
    summary_embedding = embed_model.encode([summary])
    reference_embedding = embed_model.encode([reference])
    similarity = cosine_similarity(summary_embedding, reference_embedding)[0][0]
    return similarity

def compute_extractive_density(summary, reference):
    """Compute extractive density (word overlap)."""
    summary_words = set(summary.lower().split())
    reference_words = set(reference.lower().split())
    overlap = summary_words.intersection(reference_words)
    density = len(overlap) / len(summary_words) if summary_words else 0
    return density

def compute_all_metrics(summary, reference_text, embed_model):
    """Compute all evaluation metrics."""
    rouge_scores = compute_rouge_scores(summary, reference_text)

    P, R, F1 = bert_score([summary], [reference_text], lang='en', verbose=False)

    metrics = {
        **rouge_scores,
        'bertscore_precision': float(P[0]),
        'bertscore_recall': float(R[0]),
        'bertscore_f1': float(F1[0]),
        'semantic_similarity': compute_semantic_similarity(summary, reference_text, embed_model),
        'extractive_density': compute_extractive_density(summary, reference_text)
    }

    return metrics

print("Evaluation functions defined!")

⏳ Loading embedding model for evaluation...
✅ Evaluation functions defined!


In [ ]:
# ==============================================================================
# CELL 9: COMPUTE TRADITIONAL METRICS
# ==============================================================================

print("\n" + "="*80)
print("COMPUTING TRADITIONAL METRICS")
print("="*80)

# -------------------------------------------------------------------------
# 1. EVALUATE CHUNK SUMMARY
# -------------------------------------------------------------------------
print("\nEvaluating chunk summary...")
chunk_metrics = compute_all_metrics(
    summary=chunk_result['summary'],
    reference_text=chunk_result['reference_text'],
    embed_model=eval_embed_model
)

chunk_result['metrics'] = chunk_metrics

print("\nChunk Summary Metrics:")
print(f"  ROUGE-1 F1:          {chunk_metrics['rouge1_f1']:.4f}")
print(f"  ROUGE-2 F1:          {chunk_metrics['rouge2_f1']:.4f}")
print(f"  ROUGE-L F1:          {chunk_metrics['rougeL_f1']:.4f}")
print(f"  BERTScore F1:        {chunk_metrics['bertscore_f1']:.4f}")
print(f"  Semantic Similarity: {chunk_metrics['semantic_similarity']:.4f}")
print(f"  Extractive Density:  {chunk_metrics['extractive_density']:.4f}")

# -------------------------------------------------------------------------
# 2. EVALUATE PAPER SUMMARIES
# -------------------------------------------------------------------------
print(f"\nEvaluating {len(paper_results['papers'])} paper summaries...")

for i, paper in enumerate(paper_results['papers'], 1):
    print(f"  Paper {i}/{len(paper_results['papers'])}... ", end='')

    metrics = compute_all_metrics(
        summary=paper['summary'],
        reference_text=paper['reference_text'],
        embed_model=eval_embed_model
    )

    paper['metrics'] = metrics
    print(f"Done (BERTScore: {metrics['bertscore_f1']:.3f})")

# Compute average metrics for papers
paper_summaries = paper_results['papers']
paper_avg_metrics = {}
metric_keys = ['rouge1_f1', 'rouge2_f1', 'rougeL_f1', 'bertscore_f1',
               'semantic_similarity', 'extractive_density']

for key in metric_keys:
    values = [p['metrics'][key] for p in paper_summaries]
    paper_avg_metrics[key] = np.mean(values)

print("\nAverage Paper Summary Metrics:")
for key, value in paper_avg_metrics.items():
    print(f"  {key}: {value:.4f}")

print("\n" + "="*80)
print("TRADITIONAL METRICS COMPLETE!")
print("="*80)


COMPUTING TRADITIONAL METRICS

⏳ Evaluating chunk summary...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Chunk Summary Metrics:
  ROUGE-1 F1:          0.0955
  ROUGE-2 F1:          0.0474
  ROUGE-L F1:          0.0563
  BERTScore F1:        0.7992
  Semantic Similarity: 0.5535
  Extractive Density:  0.5899

⏳ Evaluating 10 paper summaries...
  Paper 1/10... 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Done (BERTScore: 0.841)
  Paper 2/10... 

...

Done (BERTScore: 0.800)
  Paper 9/10... 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Done (BERTScore: 0.820)
  Paper 10/10... 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Done (BERTScore: 0.829)

✅ Average Paper Summary Metrics:
  rouge1_f1: 0.0608
  rouge2_f1: 0.0319
  rougeL_f1: 0.0429
  bertscore_f1: 0.8107
  semantic_similarity: 0.6839
  extractive_density: 0.6979

TRADITIONAL METRICS COMPLETE!


In [ ]:
from datetime import datetime

In [ ]:
# ==============================================================================
# CELL 10: SAVE RESULTS
# ==============================================================================

def convert_to_serializable(obj):
    """Convert numpy types to Python native types for JSON serialization."""
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, (np.floating, np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.integer, np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# Create timestamped directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
safe_query = "".join([c if c.isalnum() else "_" for c in query])[:50]
results_dir = os.path.join(OUTPUT_DIR, f"{timestamp}_{safe_query}")
os.makedirs(results_dir, exist_ok=True)

print(f"\nSaving results to: {results_dir}")

# -------------------------------------------------------------------------
# 1. Save chunk result (JSON)
# -------------------------------------------------------------------------
with open(os.path.join(results_dir, "chunk_summary_with_eval.json"), 'w') as f:
    # Remove reference_text for cleaner JSON (it's very long)
    chunk_save = {k: v for k, v in chunk_result.items() if k != 'reference_text'}
    chunk_save = convert_to_serializable(chunk_save)
    json.dump(chunk_save, f, indent=4, ensure_ascii=False)

# -------------------------------------------------------------------------
# 2. Save paper results (JSON)
# -------------------------------------------------------------------------
with open(os.path.join(results_dir, "paper_summaries_with_eval.json"), 'w') as f:
    # Remove reference_text for cleaner JSON
    papers_save = []
    for p in paper_summaries:
        p_copy = {k: v for k, v in p.items() if k != 'reference_text'}
        papers_save.append(p_copy)

    result = {
        'query': query,
        'papers': papers_save,
        'avg_metrics': paper_avg_metrics
    }
    result = convert_to_serializable(result)
    json.dump(result, f, indent=4, ensure_ascii=False)

# -------------------------------------------------------------------------
# 3. Save flattened CSV
# -------------------------------------------------------------------------
flat_records = []

# Chunk summary record
flat_records.append({
    'query': query,
    'summary_type': 'CHUNKS_ONLY',
    'doi': json.dumps(chunk_result['cited_dois']),
    'title': 'Combined Chunks',
    'summary': chunk_result['summary'],
    'summary_word_count': chunk_result['summary_word_count'],
    'retrieved_contexts': json.dumps(chunk_result['retrieved_contexts']),
    **chunk_metrics
})

# Paper summary records
for paper in paper_summaries:
    flat_records.append({
        'query': query,
        'summary_type': 'FULL_PAPER',
        'doi': paper['doi'],
        'title': paper['title'],
        'summary': paper['summary'],
        'summary_word_count': paper['summary_word_count'],
        'reranker_score': paper['reranker_score'],
        'year': paper['year'],
        'retrieved_contexts': json.dumps(paper['retrieved_contexts']),
        **paper['metrics']
    })

df = pd.DataFrame(flat_records)
df.to_csv(os.path.join(results_dir, "all_summaries_with_eval.csv"), index=False)

# -------------------------------------------------------------------------
# 4. Save RAGAS-ready JSONL
# -------------------------------------------------------------------------
ragas_records = []

# Chunk summary for RAGAS
ragas_records.append({
    'user_input': query,
    'retrieved_contexts': chunk_result['retrieved_contexts'],
    'response': chunk_result['summary']
})

# Paper summaries for RAGAS
for paper in paper_summaries:
    ragas_records.append({
        'user_input': query,
        'retrieved_contexts': paper['retrieved_contexts'],
        'response': paper['summary']
    })

# Save RAGAS-ready JSONL
ragas_file = os.path.join(results_dir, "ragas_ready.jsonl")
with open(ragas_file, 'w', encoding='utf-8') as f:
    for record in ragas_records:
        f.write(json.dumps(record, ensure_ascii=False) + '\n')

print("\nResults saved:")
print(f"   - chunk_summary_with_eval.json")
print(f"   - paper_summaries_with_eval.json")
print(f"   - all_summaries_with_eval.csv")
print(f"   - ragas_ready.jsonl")

print("\n" + "="*80)
print("SAVING COMPLETE!")
print("="*80)


📁 Saving results to: /content/drive/My Drive/Capstone Project/FINAL_FOLDER/Summary and Eval/FINAL_APPROACH_v4/results/20251126_225534_How_have_innovations___impact_of_policies_spread_t

✅ Results saved:
   - chunk_summary_with_eval.json
   - paper_summaries_with_eval.json
   - all_summaries_with_eval.csv
   - ragas_ready.jsonl

SAVING COMPLETE!


In [ ]:
# ==============================================================================
# CELL 11: RAGAS EVALUATION SETUP
# ==============================================================================

# Install RAGAS if not already installed
try:
    from ragas import evaluate
    from ragas.metrics import faithfulness, answer_relevancy
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    from datasets import Dataset
    print("RAGAS already installed")
except ImportError:
    print("Installing RAGAS...")
    !pip install -q ragas langchain-openai datasets
    from ragas import evaluate
    from ragas.metrics import faithfulness, answer_relevancy
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    from datasets import Dataset
    print("RAGAS installed")

# Set up OpenAI API
import os

# Set your API key directly
OPENAI_API_KEY = "<add_key_here>"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("OpenAI API key set")

# Initialize RAGAS components
print("\nInitializing RAGAS components...")
api_llm = ChatOpenAI(model="gpt-4o", temperature=0)
api_embeddings = OpenAIEmbeddings()

print("RAGAS setup complete!")

✅ RAGAS already installed
✅ OpenAI API key set

⏳ Initializing RAGAS components...
✅ RAGAS setup complete!


In [ ]:
# ==============================================================================
# CELL 12: RUN RAGAS EVALUATION
# ==============================================================================

print("\n" + "="*80)
print("RAGAS EVALUATION")
print("="*80)

# -------------------------------------------------------------------------
# 1. Prepare RAGAS dataset
# -------------------------------------------------------------------------
print("\nPreparing RAGAS dataset...")

ragas_data = []

# Chunk summary for RAGAS
ragas_data.append({
    'user_input': query,
    'retrieved_contexts': chunk_result['retrieved_contexts'],
    'response': chunk_result['summary']
})

# Paper summaries for RAGAS
for paper in paper_summaries:
    ragas_data.append({
        'user_input': query,
        'retrieved_contexts': paper['retrieved_contexts'],
        'response': paper['summary']
    })

# Create RAGAS dataset
ragas_dataset = Dataset.from_list(ragas_data)

print(f"Loaded {len(ragas_data)} summaries for evaluation:")
print(f"   - 1 chunk summary")
print(f"   - {len(ragas_data)-1} paper summaries")

# -------------------------------------------------------------------------
# 2. Run RAGAS evaluation
# -------------------------------------------------------------------------
print("\nRunning RAGAS evaluation...")
print("This may take a few minutes (makes API calls to OpenAI)...")

metrics_to_run = [faithfulness, answer_relevancy]

ragas_result = evaluate(
    ragas_dataset,
    metrics=metrics_to_run,
    llm=api_llm,
    embeddings=api_embeddings
)

print("RAGAS evaluation complete!")

# -------------------------------------------------------------------------
# 3. Process and display results
# -------------------------------------------------------------------------
# Convert to DataFrame
ragas_df = ragas_result.to_pandas()

# Add summary type for clarity
ragas_df['summary_type'] = ['CHUNKS_ONLY'] + ['FULL_PAPER'] * (len(ragas_df) - 1)

# Display results
print("\n" + "="*80)
print("RAGAS EVALUATION RESULTS")
print("="*80)

print("\nPer-Summary Scores:")
print(ragas_df[['summary_type', 'faithfulness', 'answer_relevancy']].to_string(index=False))

print("\n" + "="*80)
print("AVERAGE SCORES BY TYPE")
print("="*80)

# Chunk summary scores
chunk_ragas = ragas_df[ragas_df['summary_type'] == 'CHUNKS_ONLY']
print("\nChunk Summary:")
print(f"  Faithfulness:      {chunk_ragas['faithfulness'].values[0]:.4f}")
print(f"  Answer Relevancy:  {chunk_ragas['answer_relevancy'].values[0]:.4f}")

# Paper summaries average
paper_ragas = ragas_df[ragas_df['summary_type'] == 'FULL_PAPER']
print("\nPaper Summaries (Average):")
print(f"  Faithfulness:      {paper_ragas['faithfulness'].mean():.4f}")
print(f"  Answer Relevancy:  {paper_ragas['answer_relevancy'].mean():.4f}")

# -------------------------------------------------------------------------
# 4. Save RAGAS results
# -------------------------------------------------------------------------
ragas_df.to_csv(os.path.join(results_dir, "ragas_evaluation.csv"), index=False)

with open(os.path.join(results_dir, "ragas_evaluation.json"), 'w') as f:
    ragas_results = {
        'chunk_summary': {
            'faithfulness': float(chunk_ragas['faithfulness'].values[0]),
            'answer_relevancy': float(chunk_ragas['answer_relevancy'].values[0])
        },
        'paper_summaries_avg': {
            'faithfulness': float(paper_ragas['faithfulness'].mean()),
            'answer_relevancy': float(paper_ragas['answer_relevancy'].mean())
        },
        'all_results': ragas_df.to_dict('records')
    }
    json.dump(ragas_results, f, indent=4)

print(f"\nRAGAS results saved to:")
print(f"   - {os.path.join(results_dir, 'ragas_evaluation.csv')}")
print(f"   - {os.path.join(results_dir, 'ragas_evaluation.json')}")

print("\n" + "="*80)
print("ALL EVALUATIONS COMPLETE!")
print("="*80)
print(f"  Traditional metrics (ROUGE, BERTScore, etc.)")
print(f"  RAGAS metrics (Faithfulness, Answer Relevancy)")
print(f"  Evaluated separately for chunks vs papers")
print(f"\nAll results saved to: {results_dir}")


RAGAS EVALUATION

⏳ Preparing RAGAS dataset...
✅ Loaded 11 summaries for evaluation:
   - 1 chunk summary
   - 10 paper summaries

⏳ Running RAGAS evaluation...
⚠️  This may take a few minutes (makes API calls to OpenAI)...


Evaluating:   0%|          | 0/22 [00:00<?, ?it/s]

✅ RAGAS evaluation complete!

RAGAS EVALUATION RESULTS

📊 Per-Summary Scores:
summary_type  faithfulness  answer_relevancy
 CHUNKS_ONLY      0.964286          0.780524
  FULL_PAPER      1.000000          0.741591
  FULL_PAPER      1.000000          0.770849
  FULL_PAPER      0.888889          0.743182
  FULL_PAPER      0.692308          0.758346
  FULL_PAPER      1.000000          0.778299
  FULL_PAPER      1.000000          0.735569
  FULL_PAPER      1.000000          0.789979
  FULL_PAPER      0.888889          0.777979
  FULL_PAPER      0.823529          0.757191
  FULL_PAPER      1.000000          0.760712

AVERAGE SCORES BY TYPE

✅ Chunk Summary:
  Faithfulness:      0.9643
  Answer Relevancy:  0.7805

✅ Paper Summaries (Average):
  Faithfulness:      0.9294
  Answer Relevancy:  0.7614

✅ RAGAS results saved to:
   - /content/drive/My Drive/Capstone Project/FINAL_FOLDER/Summary and Eval/FINAL_APPROACH_v4/results/20251126_225534_How_have_innovations___impact_of_policies_spread_t/ra

In [ ]:
import openai
import os

# Set your API key
openai.api_key = os.environ["OPENAI_API_KEY"]

# List all available models
models = openai.models.list()

print("Available models:")
print("="*80)
for model in models.data:
    print(f"  - {model.id}")

# Filter for GPT models only
print("GPT Models:")
gpt_models = [m.id for m in models.data if 'gpt' in m.id.lower()]
for model in sorted(gpt_models):
    print(f"  - {model}")

Available models:
  - gpt-3.5-turbo
  - gpt-5.1-codex-mini
  - gpt-5.1-chat-latest
  - gpt-5.1-2025-11-13
  - gpt-5.1
  - gpt-5.1-codex
  - davinci-002
  - babbage-002
  - gpt-3.5-turbo-instruct
  - gpt-3.5-turbo-instruct-0914
  - dall-e-3
  - dall-e-2
  - gpt-3.5-turbo-1106
  - tts-1-hd
  - tts-1-1106
  - tts-1-hd-1106
  - text-embedding-3-small
  - text-embedding-3-large
  - gpt-3.5-turbo-0125
  - gpt-4o
  - gpt-4o-2024-05-13
  - gpt-4o-mini-2024-07-18
  - gpt-4o-mini
  - gpt-4o-2024-08-06
  - gpt-4o-realtime-preview-2024-10-01
  - gpt-4o-audio-preview-2024-10-01
  - gpt-4o-audio-preview
  - gpt-4o-realtime-preview
  - omni-moderation-latest
  - omni-moderation-2024-09-26
  - gpt-4o-realtime-preview-2024-12-17
  - gpt-4o-audio-preview-2024-12-17
  - gpt-4o-mini-realtime-preview-2024-12-17
  - gpt-4o-mini-audio-preview-2024-12-17
  - o1-2024-12-17
  - o1
  - gpt-4o-mini-realtime-preview
  - gpt-4o-mini-audio-preview
  - o3-mini
  - o3-mini-2025-01-31
  - gpt-4o-2024-11-20
  - gpt-4o-s